<a href="https://colab.research.google.com/github/deepika-rana/Horror-Plot-Generator/blob/main/Horror_Plot_Generator_Halloween.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Horror Plot Generator

A character-level LSTM is contructed with PyTorch to generate a horror plot on the ocassion of Halloween. The network will train character by character on some text, then generate new text character by character. 

The input used to train this model was from a public dataset on kaggle that has more than 15k movie plots.
**This model will be able to generate new text based on the text from the lyrics!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn).

In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F

## Load in Data


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = "/content/drive/My Drive/mpst_full_data.csv"
data = pd.read_csv(path)

In [4]:
genre_based = data[data['tags'].str.contains('horror','dark') == True].copy()
plot_based = data[data['plot_synopsis'].str.contains('violence','murder') == True].copy()
suspense_based = data[data['tags'].str.contains('suspenseful','story telling') == True].copy()
story_based = data[data['tags'].str.contains('mystery','cruelty') == True].copy()
fear_based = data[data['tags'].str.contains('paranormal','haunting') == True].copy()
data = pd.concat([genre_based, plot_based,suspense_based,story_based,fear_based]).drop_duplicates().reset_index(drop=True)
data = plot_based.dropna(subset=['tags'])
data[:20]

,imdb_id,title,plot_synopsis,tags,split,synopsis_source
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb
16,tt1232776,Fish Tank,"We open with Mia (Katie Jarvis), a 15 year old...","suspenseful, depressing, realism",train,imdb
57,tt2937898,A Most Violent Year,"In 1981 New York, Abel Morales (Oscar Isaac) i...",violence,train,wikipedia
73,tt2216240,Kapringen,"The film is about ancient business model, alre...","dramatic, suspenseful, mystery",train,imdb
86,tt0310793,Bowling for Columbine,In Michael Moore's discussions with various fi...,"brainwashing, violence, satire, murder, stupid",test,imdb
104,tt0088757,Avenging Angel,"Set a few years after the first 'Angel' film, ...",cult,train,imdb
136,tt0486946,Wild Hogs,"Doug Madsen (Tim Allen), Woody Stevens (John T...",humor,train,imdb
168,tt0429493,The A-Team,"The A-Team is a naturally episodic show, with ...","comedy, murder, violence, flashback, clever, p...",train,wikipedia
172,tt0074084,Novecento,Born the same day at the turn of the 20th cent...,"cruelty, murder, violence, flashback, insanity...",train,imdb


In [7]:
plot = ''.join(data.plot_synopsis)
plot = plot.lower()
plot_based = ''.join([c for c in plot if c not in "#$%&()*+,-./:;<=>@[\\]^_`{|}~"])
plot[:300]

"note: this synopsis is for the orginal italian release with the segments in this certain order.boris karloff introduces three horror tales of the macabre and the supernatural known as the 'three faces of fear'.the telephonerosy (michele mercier) is an attractive, high-priced parisian call-girl who r"

### Tokenization

In the cells, below, I'm creating a couple **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [8]:
# Creating two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(plot))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encoding the text
encoded = np.array([char2int[ch] for ch in plot])

And those same characters from above, are encoded as integers.

In [9]:
encoded[:100]

array([ 74, 113,  85,   1,  91,  34,  85,  23,  95,  40,  34,  40,  72,
        74, 113, 108,  40,  95,  40,  34,  95,  40,  34,  43, 113,  17,
        34,  85,  23,   1,  34, 113,  17,  78,  95,  74,  46,  38,  34,
        95,  85,  46,  38,  95,  46,  74,  34,  17,   1,  38,   1,  46,
        40,   1,  34, 102,  95,  85,  23,  34,  85,  23,   1,  34,  40,
         1,  78,  80,   1,  74,  85,  40,  34,  95,  74,  34,  85,  23,
        95,  40,  34,  41,   1,  17,  85,  46,  95,  74,  34, 113,  17,
        81,   1,  17,  47,  54, 113,  17,  95,  40])

## Pre-processing the data

The LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an integer (via our created dictionary) and *then* converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. One-hot encoding the data using a function.


In [10]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [11]:
# Check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


## Making training mini-batches


To train on this data, I created mini-batches for training. 

In this example, taking the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `batch_size`. Each of the sequences will be `seq_length` long.

### Creating Batches

**1. The first thing is to discard some of the text so to obtain completely full mini-batches.**

**2. After that, split `arr` into $N$ batches.** 

**3. Now that we have this array, we can iterate through it to get our mini-batches.**


In [12]:
def get_batches(arr, batch_size, seq_length):
    '''A generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    n_batches = len(arr)//batch_size_total
    
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

### Testing the Implementation

Now I'll make some data sets and we can check out what's going on as we batch data. Here, as an example, I'm going to use a batch size of 8 and 50 sequence steps.

In [13]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [14]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[ 74 113  85   1  91  34  85  23  95  40]
 [ 40   1  34  43  95  74  81  40  34  41]
 [  1   1  21  34  46  40  40 110  80   1]
 [ 34  46  43  43  46  95  17  34  46 108]
 [ 46  80   1  47  85   1  81  81  72  34]
 [ 95 108 108  38   1  23 113  17  74  75]
 [ 74  41  38 110  81  95  74  78  34  17]
 [ 95  41  34  85  17  95  54   1  75  34]]

y
 [[113  85   1  91  34  85  23  95  40  34]
 [  1  34  43  95  74  81  40  34  41  23]
 [  1  21  34  46  40  40 110  80   1  40]
 [ 46  43  43  46  95  17  34  46 108 108]
 [ 80   1  47  85   1  81  81  72  34  78]
 [108 108  38   1  23 113  17  74  75  34]
 [ 41  38 110  81  95  74  78  34  17  46]
 [ 41  34  85  17  95  54   1  75  34  54]]


---
## Defining the network with PyTorch

Next, I'll use PyTorch to define the architecture of the network. Started by defining the layers and operations. Then, defined a method for the forward pass. 

In [15]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [16]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,  
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## define the layers of the model
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        self.dropout = nn.Dropout(drop_prob)
        
        self.fc = nn.Linear(n_hidden, len(self.chars))
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        # pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        out = out.contiguous().view(-1, self.n_hidden)
        
        out = self.fc(out)
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

## Time to train

The train function gives the ability to set the number of epochs, the learning rate, and other parameters.

Below using an Adam optimizer and cross entropy loss since as looking at character class scores as output. Calculate the loss and perform backpropagation.

In [17]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Instantiating the model

Actually training the network. First creating the network itself, with some given hyperparameters. Then, defined the mini-batches sizes, and started training!

In [18]:
#Set model hyperparameters
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(116, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=116, bias=True)
)


### Set training hyperparameters!

In [ ]:
batch_size = 128
seq_length = 10
n_epochs = 50 

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/50... Step: 10... Loss: 3.1239... Val Loss: 3.0856
Epoch: 1/50... Step: 20... Loss: 3.0397... Val Loss: 3.0064
Epoch: 1/50... Step: 30... Loss: 3.0106... Val Loss: 2.9890
Epoch: 1/50... Step: 40... Loss: 2.9870... Val Loss: 2.9832
Epoch: 1/50... Step: 50... Loss: 2.9959... Val Loss: 2.9788
Epoch: 1/50... Step: 60... Loss: 3.0008... Val Loss: 2.9778
Epoch: 1/50... Step: 70... Loss: 2.9274... Val Loss: 2.9781
Epoch: 1/50... Step: 80... Loss: 2.9622... Val Loss: 2.9750
Epoch: 1/50... Step: 90... Loss: 3.0204... Val Loss: 2.9727
Epoch: 1/50... Step: 100... Loss: 2.9649... Val Loss: 2.9659
Epoch: 1/50... Step: 110... Loss: 3.0139... Val Loss: 2.9509
Epoch: 1/50... Step: 120... Loss: 2.9469... Val Loss: 2.9199
Epoch: 1/50... Step: 130... Loss: 2.9111... Val Loss: 2.8900
Epoch: 1/50... Step: 140... Loss: 2.8482... Val Loss: 2.8130
Epoch: 1/50... Step: 150... Loss: 2.7813... Val Loss: 2.7574
Epoch: 1/50... Step: 160... Loss: 2.7046... Val Loss: 2.6815
Epoch: 1/50... Step: 170... Loss:

## Checkpoint

After training, saved the model to load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [ ]:
# change the name, for saving multiple files
model_name = 'rnn_x_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Making Predictions

Now that the model is trained, I'll want to sample from it and make predictions about next characters! To sample, pass in a character and have the network predict the next character. Then take that character, pass it back in, and get another predicted character. Just keep doing this and generate a bunch of text!


The output of RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

> To actually get the next character, applied a softmax function, which gives us a *probability* distribution that can then sample to predict the next character.

### Top K sampling

The predictions come from a categorical probability distribution over all the possible characters. The sample text can be made more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving completely absurd characters while allowing it to introduce some noise and randomness into the sampled text. 

In [ ]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating the plot

Typically I'll want to prime the network to build up a hidden state. Otherwise the network will start out generating characters at random which will not generate a decent horror plot. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [ ]:
def PlotGenerate(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

##Plot Generation by the Neural Network

Outputs

In [ ]:
print(PlotGenerate(net, 1000, prime='A girl hanging from a tree upside down', top_k=5))

scenes to stand them they have been a mother that, but she should take him to the tribe to heal both mare, and this is being restoning their apartment and he has no interview on thinking that the season two sent the screen as he could step for too clate to the car. she says they are standing on tom and tells him that he's an american for the tournament, and says she has no interesting the police decision in a metaphor.is talking as his wife soon of the same time. the scortion thinks suspicious. the second to gathing her featured, but tarek says that he had a marriage, still hugh alerand (a fearing of closer and starts to talk about how to get a marine and still thinks that this is bringing target in the team about leging the prisoners. they start to get help off at his still, as the two of them drives off the courter that is freelan from the first manish that he has burned her for the county and son it is being striggering her time he sees her way for a child. a man named seems to bang

In [ ]:
print(PlotGenerate(net, 500, prime='Crippled person possessed by haunted spirits', top_k=10))

In [ ]:
print(PlotGenerate(net, 1200, prime='Cawing sounds echo the mountains', top_k=5))